In [ ]:
import xarray as xr
import pandas as pd
import panel as pn
import numpy as np
import panel as pn
from holoviews import opts
import holoviews as hv
from thredds_crawler.crawl import Crawl
import datetime as dt
from functools import partial
from get_data import get_urls, get_model_data, get_obs_data

In [ ]:
pn.extension()
hv.extension("bokeh")

# Use-case: Scatter 1

## Introduction 

This notebook contains the code for generating a scatter plot of model vs. observational data for a chosen site, model, variable and time.

The code for fetching the data can be found in gt_data.py 

## Code 

In [ ]:
def merge_data(var_obs,var_mod,variable):
    #rename column names to easier differentiate between values when combined
    var_obs = var_obs.rename(columns={variable:"var_obs"})
    var_mod = var_mod.rename(columns={variable:"var_mod"})
    
    #merges the dataframes based on matching time points, gets rid of all mismatches for time
    if len(var_obs.index) < len(var_mod):
        merged = var_obs.merge(var_mod, left_index=True, right_index=True, how='inner')
    else:
        merged = var_mod.merge(var_obs, left_index=True, right_index=True, how='inner')

    return merged

In [ ]:
#plots the data as a scatter plot with a linear regression 
#axies are kept constant 
def plot_scatter(data,var,line_reg,one_one):
    
    scatter = hv.Points(data)
    
    if line_reg:
        linereg = hv.Slope.from_scatter(scatter)
        linereg.opts(color = 'red')
    
        scatter_1 = scatter * linereg
    elif one_one:
        line = hv.Slope(1,0)
        
        scatter_1 = scatter*line
    else:
        scatter_1 = scatter
    
    bounds_upper = max(data.max().var_obs,data.max().var_mod)
    bounds_lower = min(data.min().var_obs,data.min().var_mod)
    
    bounds = (bounds_lower,bounds_upper)
     
    scatter_1.opts(title="Scatter 1: "+var+' '+data.index[0].strftime("%Y-%-m-%d")+'-'+data.index[-1].strftime("%Y-%-m-%d"),
        ylabel="model",
        xlabel="observation",
        xlim = bounds,
        ylim = bounds,
        width=400,
        height=400
    )
    
    return scatter_1

In [ ]:
#user selection variable
variable = pn.widgets.Select(
    name="Variables", options=['tas','snd'], margin=(0, 20, 0, 0)
)

#user selection start and end date only sop 1 as of right now
dates = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2018, 2, 1), end=dt.datetime(2018, 3, 31),
    value=(dt.datetime(2018, 2, 1), dt.datetime(2018, 2, 4))
)

#user selection start time for model 
start_time = pn.widgets.Select(
    name="Start time", options=['00','12'], margin=(0, 20, 0, 0)
)

#user selection of which day should be fetched from the model files
#for concat
concat_day = pn.widgets.Select(
    name="Model day selected", options={1:0,2:1,3:2}, margin=(0,20,0,0)
)

#TO DO: add dependecy so that 1-1 toggles line reg off and vice versa
line_reg = pn.widgets.Checkbox(name='Linear regression')
one_one = pn.widgets.Checkbox(name='1 to 1 line')

In [ ]:
@pn.depends(variable,dates,start_time,concat_day,line_reg,one_one)
def scatter_plot(variable,dates,start_time,concat_day,line_reg,one_one):
    urls_mod, urls_obs = get_urls(dates[0],dates[1],start_time,"ifs-ecmwf","sodankyla",variable,concat_day)
    
    out_type='concatenated'
    var_mod = get_model_data(out_type,urls_mod,concat_day)
    var_obs = get_obs_data(urls_obs,dates[0],dates[1],variable)
    
    merged = merge_data(var_obs,var_mod,variable)
    
    plot = plot_scatter(merged,variable,line_reg,one_one)
    return plot

In [ ]:
pn.Column("scatter 1 sop 1", scatter_plot,pn.Row(pn.Column(variable,start_time),pn.Column(dates,concat_day),pn.Column(one_one,line_reg)),width_policy="max").servable()